# Importing Libraries

In [1]:
import pandas as pd
import pyarrow
import numpy as np

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

# Reading Data

In [3]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')

## Q_1

In [4]:
df_jan.head()
df_jan.shape

(2964624, 19)

In [5]:
df_feb.shape

(3007526, 19)

In [6]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
df_jan['duration'].describe()

count    2.964624e+06
mean     1.561295e+01
std      3.485105e+01
min     -1.356667e+01
25%      7.150000e+00
50%      1.163333e+01
75%      1.868333e+01
max      9.455400e+03
Name: duration, dtype: float64

## Q_2

In [10]:
df_jan['duration'].std()

34.851053592192876

## Q_3

In [11]:
df_jan['outlier'] = df_jan.duration.apply(lambda x: 1 if x < 1 or x > 60 else 0)

df_jan.outlier.value_counts(dropna= False, normalize= True)

0    0.977833
1    0.022167
Name: outlier, dtype: float64

In [12]:
df_feb['outlier'] = df_feb.duration.apply(lambda x: 1 if x < 1 or x > 60 else 0)

df_feb.outlier.value_counts(dropna= False, normalize= True)

0    0.976903
1    0.023097
Name: outlier, dtype: float64

In [13]:
df_jan['month'] = 'jan'
df_feb['month'] = 'feb'

In [14]:
df = pd.concat([df_jan, df_feb])
df.shape

(5972150, 22)

In [15]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [35]:
df['duration'].std()

10.205600634889613

In [31]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'duration',
       'outlier'],
      dtype='object')

# Building Model 

## Q_4,5

In [16]:
train_dicts = df[df.month == 'jan'][categorical ].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(X_train.shape)
target = 'duration'
y_train = df[df.month == 'jan'][target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

(2898906, 518)


7.94617335953716

## Q_6

In [26]:
test_dicts = df[df.month == 'feb'][categorical ].to_dict(orient='records')

X_test = dv.transform(test_dicts)

print(X_test.shape)
target = 'duration'
y_test= df[df.month == 'feb'][target].values



y_test_pred = lr.predict(X_test)

mean_squared_error(y_test, y_test_pred, squared=False)

(2938060, 518)


8.123382959638906

261